# Calculating B-S Greeks 


In [19]:
import numpy as np
from scipy.stats import norm
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho


In [20]:
# Implementation of Black-Scholes formula

# Define variables
r = 0.01
S = 30
K = 40
T = 240/365
sigma = 0.30

def blackScholes(r, S, K, T, sigma, type = "c"):
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    try:
        if type == "c":
            price = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            price = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
        return price, bs(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")
    

In [21]:
print("Option Price: ", blackScholes(r,S,K,T,sigma,"c"))

Option Price:  (0.5132843798399405, 0.5132843798399411)


# Delta
Delta measures the rate of change of the theoretical option value with respect to the change of the underlying S.  
Delta measures the sensitivity of an option to a shift in the underlying asset, essentially describing how much the price of the former will change for a given move in the price of the latter.  
Delta is also commonly regarded as an indication of the probability, at any given point in time, that the option will expire in the money.  
  
$\Phi$ for cdf  
  
$\Delta = \frac{\partial V}{\partial S}$  
$\Delta_{call} = \Phi(d1)$  
$\Delta_{put} = -\Phi(-d1)$

In [23]:
def delta_calc(r, S, K, T, sigma, type = "c"):
    "Calculate delta of an option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    
    try:
        if type == "c":
            delta_calc = norm.cdf(d1, 0, 1)
        elif type == "p":
            delta_calc = -norm.cdf(-d1, 0, 1)
        return delta_calc, delta(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")
    

# Gamma  
Delta is only useful in determining an option’s sensitivity to small movements in the underlying, because it doesn’t stay constant. The option risk parameter which measures the rate by which delta changes with shifts in the underlying price is called gamma.  

Long options have positive gamma, while short options have negative. Gamma is at its highest for at-the-money and close-to-expiry options.  
   
$\phi$ for pdf
  
$\Gamma = \frac{\partial \delta}{\partial S} =  \frac{\partial^2 V}{\partial S^2}$  
$\Gamma = \frac{\phi(d1)}{S\sigma \sqrt{\tau}}$

In [22]:
def gamma_calc(r, S, K, T, sigma, type = "c"):
    "Calculate gamma of a option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    try:
        gamma_calc = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T)) 
            
        return gamma_calc, gamma(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")
    

# Vega
Although vega isn’t actually a Greek letter, it’s one of the most important “Greeks” in options trading; it measures the option’s sensitivity to changes in its implied volatility, typically for a move of 1% in the latter.

As with gamma, long options have positive vega, while short options have negative. Contrary to the case for gamma, though, the more time there remains until the option’s expiry, the greater its vega will be. Vega is thus one of the important risk considerations for longer-dated options. 
  
In practice, we want to see what is sensitivity to the 1% change in volatilty same for rho
  
$\upsilon = \frac{\partial V}{\partial \sigma}$  
$\upsilon = S\phi(d1)\sqrt(\tau)$


In [33]:
def vega_calc(r, S, K, T, sigma, type = "c"):
    "Calculate vega of a option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    try:
        vega_calc = S*norm.pdf(d1, 0, 1)*np.sqrt(T) 
            
        return vega_calc*0.01, vega(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")
    

# Theta  
Theta measures the sentivity of the value of the derivative to the passage of time.  

Theta is the risk parameter used to describe the time decay in the option’s value. It’s typically expressed as the loss the option will suffer for one calendar day. For that reason, options are often quoted lower on Fridays, in order to incorporate the effects of three days’ worth of time value about to be lost.

Theta can be thought of as a trade-off for gamma. Long options have negative theta, and short options have positive. 

We want to see the sensitivity per day. => /365
  
$\Theta = -\frac{\partial V}{\partial \tau}$  
$\Theta_{call} = -\frac{S\phi(d1)\sigma}{2\tau} - rKexp(-rT)\Phi(d2)$  
$\Theta_{put} = -\frac{S\phi(d1)\sigma}{2\tau} + rKexp(-rT)\Phi(-d2)$

In [40]:
def theta_calc(r, S, K, T, sigma, type = "c"):
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    try:
        if type == "c":
            theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        return theta_calc/365, theta(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")
    

# Rho  
Rho measures the sentivity to the interest rate.  
  
Rho is generally considered the least important of the most commonly followed Greeks. It describes an option’s sensitivity to changes in the interest rate used to derive the option’s value.

An increase in interest rates will increase the underlying’s forward price, and will therefore be good for call options and bad for puts. The former therefore have positive rho, while the latter negative.  
  
In practice, we want to see what is sensitivity to the 1% change in interest rate.

  
$\rho = \frac{\partial V}{\partial r}$  
$\rho_{call} = K\tau exp(-rT)\Phi(d2)$  
$\rho_{put} = K\tau exp(-rT)\Phi(-d2)$  


In [26]:
def rho_calc(r, S, K, T, sigma, type="c"):
    "Calculate rho of an option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            rho_calc = K*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            rho_calc = -K*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        return rho_calc*0.01, rho(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

# All together

In [41]:
option_type = 'c'  
  
print("Option Price: ", [round(x,3) for x in blackScholes(r, S, K, T, sigma, option_type)])
print("       Delta: ", [round(x,3) for x in delta_calc(r, S, K, T, sigma, option_type)])
print("       Gamma: ", [round(x,3) for x in gamma_calc(r, S, K, T, sigma, option_type)])
print("       Vega : ", [round(x,3) for x in vega_calc(r, S, K, T, sigma, option_type)])
print("       Theta: ", [round(x,3) for x in theta_calc(r, S, K, T, sigma, option_type)])
print("       Rho  : ", [round(x,3) for x in rho_calc(r, S, K, T, sigma, option_type)])


Option Price:  [0.513, 0.513]
       Delta:  [0.151, 0.151]
       Gamma:  [0.032, 0.032]
       Vega :  [0.057, 0.057]
       Theta:  [-0.004, -0.004]
       Rho  :  [0.026, 0.026]
